# This NoteBook add latitude longitude To The DataFrame.

#### Import Lib's.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import lxml
from bs4 import BeautifulSoup

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Read CSV Geospatial

In [3]:
geospatial = pd.read_csv("http://cocl.us/Geospatial_data", index_col=0)
geospatial.shape

(103, 2)

#### Create data frame of wikipedia torento postal codes

In [7]:
wiki_html_page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup_object = BeautifulSoup(wiki_html_page, "lxml")

# Table tag is: <table class="wikitable">
html_table = soup_object.find("table", class_="wikitable")

all_td = html_table.findAll("td")
print("Len of the all_td: ", len(all_td))

postalCode=[]
borough=[]
nieghberhood=[]

# create list of data for each column of data frame.
for data in range(0, 539, 3):
    
    postalCode.append(all_td[data].text)
    borough.append(all_td[data+1].text)
    nieghberhood.append(all_td[data+2].text)
    
print("len postalCode {}, borough {}, nie {}".format(len(postalCode), len(borough), len(nieghberhood)))


# create torento data frame.
df_torento_neighberhood = pd.DataFrame({"PostalCode": postalCode,
                                                          "Borough": borough,
                                                          "Neighberhood": nieghberhood
                                                         })

print("shape of the torento neighberhood Data Frame: ", df_torento_neighberhood.shape)

# drop all rows have "Not assigned\n" value for column "Borough"
indexs = df_torento_neighberhood[ df_torento_neighberhood['Borough'] == "Not assigned\n" ].index

df_torento_neighberhood_clean_data = df_torento_neighberhood.drop(indexs)
df_torento_neighberhood_clean_data.reset_index(drop=True, inplace=True)

# strings have extra "\n" we replave is with ""
df_torento_neighberhood_clean_data["Neighberhood"].replace(to_replace=r'/', value=',', regex=True, inplace=True)
df_torento_neighberhood_clean_data["Neighberhood"].replace(to_replace=r'\n', value='', regex=True, inplace=True)
df_torento_neighberhood_clean_data["Borough"].replace(to_replace=r'\n', value='', regex=True, inplace=True)
df_torento_neighberhood_clean_data["PostalCode"].replace(to_replace=r'\n', value='', regex=True, inplace=True)

# check if we have null column
# print(df_torento_neighberhood_drop_no_assighned.isnull())
print("Clean Data Frame Shape: ", df_torento_neighberhood_clean_data.shape)
df_torento_neighberhood_clean_data.head()

Len of the all_td:  540
len postalCode 180, borough 180, nie 180
shape of the torento neighberhood Data Frame:  (180, 3)
Clean Data Frame Shape:  (103, 3)


,PostalCode,Borough,Neighberhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [8]:
geospatial.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


#### Create Columns for Latitude and Longitude.

In [19]:
# Sort our df_torento_neighberhood_clean_data by column PostalCode
df_torento_neighberhood_clean_data.sort_values(by="PostalCode", inplace=True)
df_torento_neighberhood_clean_data.reset_index(drop=True, inplace=True)
df_torento_neighberhood_clean_data.head()

latitude = geospatial["Latitude"].values
longitude = geospatial["Longitude"].values

df_torento_neighberhood_clean_data["Latitude"] = latitude
df_torento_neighberhood_clean_data["Longitude"] = longitude
print(df_torento_neighberhood_clean_data.shape)
df_torento_neighberhood_clean_data.head()


(103, 5)


,PostalCode,Borough,Neighberhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
